## Full likelihood

In this notebook we show some example syntax for the new 'full likelihood'. The `likelihood` object remains pretty much the same, but here we use a new `FullTheory` object that replaces `LyaTheory` which deals with the mapping from a set of likelihood parameters (`like_params=[..]`) to a set of emulator calls and a $P_{1\rm{D}}$ in velocity units. The `FullTheory` object works in Planck units, i.e. $\omega_b^2$, $\omega_c^2$, $A_s$, $n_s$ so far, we can add more later on. It uses `CAMB` to map these parameters directly into a set of emulator calls $\Delta^2_p$, $n_p$, etc at each $z$ where we have data.

NB that there is a bit of duplicate code in this iteration which can be tidied up eventually, in that the syntax I use to fit the emulator calls is also found in `fit_linP`, however there the comoving linear power is calculated at each $z$ independently, and I thought it might be faster to give all $z$s to `CAMB` to calculate at once. If it is faster this way, this will matter when it comes to sampling. The other point to note is that I do not currently calculate $f_p$, which is done in the `fit_linP` fits, and I do that as we have not yet trained the emulator on it so I do not yet want to slow down likelihood evaluations calculating an expensive quantitity that is not used.

Also to avoid calling `CAMB` twice in different parts of the code, `get_emulator_calls()` now also has an option `return_M_of_z` which will give $M(z)$ for each emulator call, which is then used to convert the comoving $P_{1\rm{D}}$ into velocity units.

First we set up a `data` object and an emulator as normal.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 140
mpl.rcParams['figure.dpi'] = 140
# our modules
import data_MPGADGET
import p1d_arxiv
import gp_emulator
import likelihood
import likelihood_parameter
import lya_theory
import full_theory
import camb_cosmo
import CAMB_model
import fit_linP

### Setup mock data, from one of simulations in the suite

In [2]:
# specify simulation suite and P1D mesurements
basedir="/p1d_emulator/sim_suites/Australia20/"
skewers_label='Ns500_wM0.05'
p1d_label=None
# specify simulation to use to generate synthetic data
test_sim_num=15
data=data_MPGADGET.P1D_MPGADGET(sim_number=test_sim_num,
                                basedir=basedir,
                                skewers_label=skewers_label,
                                data_cov_factor=1)

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


### Setup archive of P1D measurements from simulations and train the emulator

In [3]:
# specify redshift range
z_max=4
# do not use test_sim_num that was used in generating mock data
arxiv=p1d_arxiv.ArxivP1D(basedir=basedir,drop_sim_number=test_sim_num,
                            drop_tau_rescalings=True,z_max=z_max,
                            drop_temp_rescalings=True,skewers_label=skewers_label)

In [4]:
# specify parameters to be used in emulator
paramList=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# specify k range
kmax_Mpc=8
# setup GP emulator
emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,z_max=z_max,
                                verbose=False,paramList=paramList,train=True,
                                emu_type="k_bin",passArxiv=arxiv,kmax_Mpc=kmax_Mpc,
                                drop_tau_rescalings=True,
                                drop_temp_rescalings=True)

 /Users/font/Programs/Others/anaconda3/lib/python3.6/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 270 points
Optimised


### Setup and compare theory objects

Start by setting up an old theory object, that uses only compressed linP parameters

In [5]:
old_theory=lya_theory.LyaTheory(zs=data.z,emulator=emu)

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


Setup now a new theory object, that holds a CAMB object

In [6]:
new_theory=full_theory.FullTheory(zs=data.z,emulator=emu)

In [7]:
# compare emulator calls at one of the redshifts
old_emu_calls=old_theory.get_emulator_calls()
new_emu_calls=new_theory.get_emulator_calls(return_M_of_z=False)
print("z=2 emulator calls for old theory = ", old_emu_calls[0])
print("z=2 emulator calls for new theory = ", new_emu_calls[0])

Note: redshifts have been re-sorted (earliest first)
z=2 emulator calls for old theory =  {'Delta2_p': 0.6300728196477859, 'n_p': -2.2994793242105556, 'alpha_p': -0.2163759500235969, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new theory =  {'Delta2_p': 0.6300510342450939, 'n_p': -2.299445461757138, 'alpha_p': -0.2165269830826664, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}


Note that the numbers above are not supposed to exactly match, since kp_Mpc is different in both theories

### Compare now emulator calls for a different test cosmology (1st, failed attempt)

In [8]:
As=1.9e-9
ns=0.92
test_cosmo=camb_cosmo.get_cosmology(As=As,ns=ns)
test_camb_model=CAMB_model.CAMBModel(zs=data.z,cosmo=test_cosmo,verbose=False)

In [9]:
old_test_theory=lya_theory.LyaTheory(zs=data.z,emulator=emu,cosmo_fid=test_cosmo,verbose=False)

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


In [10]:
new_test_theory=full_theory.FullTheory(zs=data.z,emulator=emu,camb_model_fid=test_camb_model,verbose=False)

In [11]:
# compare emulator calls at one of the redshifts
old_test_emu_calls=old_test_theory.get_emulator_calls()
new_test_emu_calls=new_test_theory.get_emulator_calls(return_M_of_z=False)
print("z=2 emulator calls for old test theory = ", old_test_emu_calls[0])
print("z=2 emulator calls for new test theory = ", new_test_emu_calls[0])

Note: redshifts have been re-sorted (earliest first)
z=2 emulator calls for old test theory =  {'Delta2_p': 0.5065581715009833, 'n_p': -2.344479324210556, 'alpha_p': -0.2163759500235972, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new test theory =  {'Delta2_p': 0.6300510342450939, 'n_p': -2.299445461757138, 'alpha_p': -0.2165269830826664, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}


The test did not work because the new theory (full_theory) does not use the input cosmology in get_emulator_calls. We probably want to change this behavior.

### Compare now emulator calls for a different test cosmology (2nd, successful attempt)

Now we will redo the test, but instead of using the test cosmology to define the fiducial cosmologies, we will treat it as a point in the sampling

In [12]:
# print compressed linP parameters from the fiducial cosmology in the old theory
old_linP=fit_linP.LinearPowerModel(cosmo=old_theory.cosmo.cosmo_fid)
old_like_params=old_linP.get_likelihood_parameters()
for par in old_like_params:
    print(par.info_str())

g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895


In [13]:
# print compressed linP parameters from the test cosmology
test_linP=fit_linP.LinearPowerModel(cosmo=test_cosmo)
old_test_like_params=test_linP.get_likelihood_parameters()
for par in old_test_like_params:
    print(par.info_str())

g_star = 0.9677508579459803
f_star = 0.9813695578403181
Delta2_star = 0.28957717426109336
n_star = -2.344335667257508
alpha_star = -0.21652703712062632


In [14]:
# call the old theory, but using the freshly computed likelihood parameters from the test cosmology
old_test_emu_calls=old_theory.get_emulator_calls(like_params=old_test_like_params)

In [15]:
# print cosmology parameters from the new theory for the test cosmology
new_like_params=new_theory.camb_model_fid.get_likelihood_parameters()
for par in new_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 2.1e-09
ns = 0.965


In [16]:
# print cosmology parameters from new test camb model
new_test_like_params=test_camb_model.get_likelihood_parameters()
for par in new_test_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 1.9e-09
ns = 0.92


In [17]:
# call the new theory, but using the freshly computed likelihood parameters for the test cosmology
new_test_emu_calls=new_theory.get_emulator_calls(like_params=new_test_like_params,return_M_of_z=False)

Note: redshifts have been re-sorted (earliest first)


In [18]:
print("z=2 emulator calls for old test theory = ", old_test_emu_calls[0])
print("z=2 emulator calls for new test theory = ", new_test_emu_calls[0])

z=2 emulator calls for old test theory =  {'Delta2_p': 0.5065711059912054, 'n_p': -2.3444793242105644, 'alpha_p': -0.21637595002353538, 'f_p': 0.9583569136684356, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new test theory =  {'Delta2_p': 0.5065420179183718, 'n_p': -2.3444454617571386, 'alpha_p': -0.21652698308265753, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}


Both new and old theories make the same emulator call for a theory that is not the fiducial!

### Test the likelihood objects with old and new theories

In [19]:
## Set up a likelihood object with the old theory object
old_free_params=['Delta2_star','n_star']
old_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=old_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

likelihood setup with 2 free parameters
2 free parameters


In [20]:
## Set up a likelihood object with the new theory object
new_free_params=["As","ns"]
new_like=likelihood.Likelihood(data=data,theory=new_theory,
                            free_parameters=new_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

likelihood setup with 2 free parameters
2 free parameters


In [21]:
# figure out values of parameters in unit cube of old theory
old_values=[]
for parname in old_free_params:
    for p in old_like_params:
        if p.name==parname:
            old_values.append(p.value_in_cube())
print(old_values)

[0.7345060352427981, 0.5066433274250222]


In [22]:
old_like.get_log_like(values=old_values,ignore_log_det_cov=False)

got P1D from emulator
compute chi2 for z=2.0
added 0.20673629807690475 to log_like
compute chi2 for z=2.25
added -4.839080429252032 to log_like
compute chi2 for z=2.5
added -5.156546576796529 to log_like
compute chi2 for z=2.75
added 6.545672499953245 to log_like
compute chi2 for z=3.0
added 14.737390215043101 to log_like
compute chi2 for z=3.25
added 12.469957201833417 to log_like
compute chi2 for z=3.5
added 4.4878427972300585 to log_like
compute chi2 for z=3.75
added -13.852877393288573 to log_like
compute chi2 for z=4.0
added -51.00686181445442 to log_like


-36.40776720165483

In [23]:
# figure out values of parameters in unit cube of new theory
new_values=[]
for parname in new_free_params:
    for p in new_like_params:
        if p.name==parname:
            new_values.append(p.value_in_cube())
print(new_values)

[0.500000000000001, 0.5]


In [24]:
new_like.get_log_like(values=new_values,ignore_log_det_cov=False)

Note: redshifts have been re-sorted (earliest first)
got P1D from emulator
compute chi2 for z=2.0
added 0.1976187610996618 to log_like
compute chi2 for z=2.25
added -4.855247715855395 to log_like
compute chi2 for z=2.5
added -5.18119514049863 to log_like
compute chi2 for z=2.75
added 6.5045018796464795 to log_like
compute chi2 for z=3.0
added 14.710400086327803 to log_like
compute chi2 for z=3.25
added 12.430543275279808 to log_like
compute chi2 for z=3.5
added 4.453444056455618 to log_like
compute chi2 for z=3.75
added -13.863758256651284 to log_like
compute chi2 for z=4.0
added -51.03583139179855 to log_like


-36.639524445994496